In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
df = pd.read_csv("dataset.csv", dtype={"Image_Path": str, "Label": str})

images_folder = "/content/images"

df['Full_Image_Path'] = images_folder + '/' + df['Image_Path'].str.replace('\\', '/')

full_image_paths = df['Full_Image_Path']
labels = df['Label']

<ipython-input-6-d12d01c69af2>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Full_Image_Path'] = images_folder + '/' + df['Image_Path'].str.replace('\\', '/')


In [14]:
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import numpy as np

# Load CSV file
df = pd.read_csv("dataset.csv")

# Load and preprocess images
images = []
labels = []

images_folder = "/content/images"

df['Full_Image_Path'] = images_folder + '/' + df['Image_Path'].str.replace('\\', '/')


<ipython-input-14-fc8bcb5773f4>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Full_Image_Path'] = images_folder + '/' + df['Image_Path'].str.replace('\\', '/')


In [16]:
from google.colab import files

uploaded = files.upload()

Saving dataset.zip to dataset.zip


In [17]:
import zipfile
import io

zip_filename = "dataset.zip"

with zipfile.ZipFile(io.BytesIO(uploaded[zip_filename]), 'r') as zip_ref:
    zip_ref.extractall("images")

In [15]:
df.head()

,Image_Path,Label,Full_Image_Path
0,dataset\American Black Duck\abd1.jpg,Anas rubripes,/content/images/dataset/American Black Duck/ab...
1,dataset\American Black Duck\abd10.jpg,Anas rubripes,/content/images/dataset/American Black Duck/ab...
2,dataset\American Black Duck\abd11.jpg,Anas rubripes,/content/images/dataset/American Black Duck/ab...
3,dataset\American Black Duck\abd12.jpg,Anas rubripes,/content/images/dataset/American Black Duck/ab...
4,dataset\American Black Duck\abd13.jpg,Anas rubripes,/content/images/dataset/American Black Duck/ab...


In [23]:
from sklearn.preprocessing import LabelEncoder

for index, row in df.iterrows():
    img_path = row['Full_Image_Path']
    label = row['Label']

    img = load_img(img_path, target_size=(299, 299))
    img_array = img_to_array(img) / 255.0  # Normalize pixel values

    images.append(img_array)
    labels.append(label)

X = np.array(images)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [24]:
# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
'''
import os
from sklearn.model_selection import train_test_split

X_train_paths, X_val_paths, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)

def preprocess_image(file_path):
    img = load_img(file_path, target_size=(299, 299))
    img_array = img_to_array(img) / 255.0
    return img_array

# Training images
X_train = np.array([preprocess_image(os.path.join(images_folder, path.replace('\\', '/'))) for path in X_train_paths])

# Validation images
X_val = np.array([preprocess_image(os.path.join(images_folder, path.replace('\\', '/'))) for path in X_val_paths])
'''

"\nimport os\nfrom sklearn.model_selection import train_test_split\n\nX_train_paths, X_val_paths, y_train, y_val = train_test_split(image_paths, labels, test_size=0.2, random_state=42)\n\ndef preprocess_image(file_path):\n    img = load_img(file_path, target_size=(299, 299))\n    img_array = img_to_array(img) / 255.0\n    return img_array\n\n# Training images\nX_train = np.array([preprocess_image(os.path.join(images_folder, path.replace('\\', '/'))) for path in X_train_paths])\n\n# Validation images\nX_val = np.array([preprocess_image(os.path.join(images_folder, path.replace('\\', '/'))) for path in X_val_paths])\n"

In [25]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, Model

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

In [26]:
from tensorflow.keras.models import Sequential

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    #layers.Dense(64, activation='relu'),  # Add another dense layer
    #layers.Dropout(0.5),  # Add another dropout layer
    layers.Dense(10, activation='softmax')
])


# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Added another dense and dropout layer, because i was noticing an overfitting issue. The accuracy while training the model was >90%, but after evaluating with the test data, the accuracy was only 11%.

In [27]:
# Training the model this way with our larger dataset consumes too much RAM and crashes Google colab's runtime

history = model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_val, y_val)
)


Epoch 1/10
10/10 [==============================] - 21s 642ms/step - loss: 2.2453 - accuracy: 0.2156 - val_loss: 1.7759 - val_accuracy: 0.4125
Epoch 2/10
10/10 [==============================] - 2s 196ms/step - loss: 1.6196 - accuracy: 0.4250 - val_loss: 1.4205 - val_accuracy: 0.6000
Epoch 3/10
10/10 [==============================] - 2s 197ms/step - loss: 1.3175 - accuracy: 0.5750 - val_loss: 1.1301 - val_accuracy: 0.7125
Epoch 4/10
10/10 [==============================] - 2s 197ms/step - loss: 1.0831 - accuracy: 0.6781 - val_loss: 0.9640 - val_accuracy: 0.8375
Epoch 5/10
10/10 [==============================] - 2s 197ms/step - loss: 0.8997 - accuracy: 0.7250 - val_loss: 0.8835 - val_accuracy: 0.8125
Epoch 6/10
10/10 [==============================] - 2s 178ms/step - loss: 0.7739 - accuracy: 0.8000 - val_loss: 0.7378 - val_accuracy: 0.8875
Epoch 7/10
10/10 [==============================] - 2s 196ms/step - loss: 0.6988 - accuracy: 0.8125 - val_loss: 0.7012 - val_accuracy: 0.8375
Epoch

In [29]:
test_loss, test_acc = model.evaluate(X_val, y_val)

print ('Test loss: {}, Test accuracy: {}'.format(test_loss, test_acc*100))

3/3 [==============================] - 0s 110ms/step - loss: 0.4914 - accuracy: 0.9125
Test loss: 0.4914044439792633, Test accuracy: 91.25000238418579


In [39]:
scientific_name_map = {
    0: 'Anas rubripes',
    1: 'Melanitta americana',
    2: 'Bucephala albeola',
    3: 'Aythya valisineria',
    4: 'Bucephala clangula',
    5: 'Aythya marila',
    6: 'Clangula hyemalis',
    7: 'Mergus serrator',
    8: 'Melanitta perspicillata',
    9: 'Melanitta deglandi'
}

# Use model.predict() to obtain predictions
predictions = model.predict(X_val)

predicted_classes = np.argmax(predictions, axis=1)

print("Unique values in predicted_classes:", np.unique(predicted_classes))

# Print the predicted scientific names for each image
for i in range(len(df)):
    predicted_scientific_name = scientific_name_map[predicted_classes[i]]
    actual_scientific_name = df['Label'].iloc[i]
    print(f"Image: {df['Image_Path'].iloc[i]}, Predicted Scientific Name: {predicted_scientific_name}, Actual Scientific Name: {actual_scientific_name}")


3/3 [==============================] - 0s 149ms/step
Unique values in predicted_classes: [0 1 2 3 4 5 6 7 8 9]
Image: dataset\American Black Duck\abd1.jpg, Predicted Scientific Name: Anas rubripes, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd10.jpg, Predicted Scientific Name: Bucephala clangula, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd11.jpg, Predicted Scientific Name: Anas rubripes, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd12.jpg, Predicted Scientific Name: Anas rubripes, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd13.jpg, Predicted Scientific Name: Bucephala clangula, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd14.jpg, Predicted Scientific Name: Bucephala clangula, Actual Scientific Name: Anas rubripes
Image: dataset\American Black Duck\abd15.jpg, Predicted Scientific Name: Aythya marila, Actual Scientific Name: Anas ru

IndexError: ignored

In [38]:
print("Length of DataFrame (df):", len(df))
print("Length of X_val:", len(X_val))
print("Length of predicted_classes:", len(predicted_classes))

Length of DataFrame (df): 200
Length of X_val: 80
Length of predicted_classes: 80


In [ ]:
df.columns

Index(['Image_Path', 'Label', 'Full_Image_Path'], dtype='object')

In [ ]:
# Approach using ImageDataGenerator provided by Keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=images_folder,
    x_col="Full_Image_Path",
    y_col="Label",
    target_size=(299, 299),
    batch_size=64,
    class_mode="sparse",
    subset="training"
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=images_folder,
    x_col="Full_Image_Path",
    y_col="Label",
    target_size=(299, 299),
    batch_size=64,
    class_mode="sparse",
    subset="validation"
)

validation_generator_test = datagen.flow_from_dataframe(
    dataframe=df,
    directory=images_folder,
    x_col="Full_Image_Path",
    y_col="Label",
    target_size=(299, 299),
    batch_size=64,
    class_mode="sparse",
    subset="validation"
)

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 200 invalid image filename(s) in x_col="Full_Image_Path". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 200 invalid image filename(s) in x_col="Full_Image_Path". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                        

In [ ]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

ValueError: ignored

In [ ]:
test_loss, test_acc = model.evaluate(validation_generator)

print ('Test loss: {}, Test accuracy: {}'.format(test_loss, test_acc*100))

ValueError: ignored

In [ ]:
import matplotlib.pyplot as plt

# Display the images
'''
for image_path in df["Full_Image_Path"]:

    plt.imshow(plt.imread(image_path))

    plt.show()
'''

In [ ]:
records, features = df.shape
print("Number of records:", records)
print("Number of features:", features)

Number of records: 5818
Number of features: 2
